# Representing Species in RMG

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import rmgpy.data.thermo
import rmgpy.species
import rmgpy.molecule

## RMG Species Class:

- contains information about which atoms are bonded and the bond orders
- bond information can be specified with SMILES, InChI, or an adjacency list
- species has an attribute for thermo, usually a NASA polynomial

Use the website to search for molecules and create adjacency lists: https://rmg.mit.edu/molecule_search

Code for the class can be found here: https://github.com/ReactionMechanismGenerator/RMG-Py/blob/main/rmgpy/species.py

## Adjacency List

Documentation: http://reactionmechanismgenerator.github.io/RMG-Py/reference/molecule/adjlist.html

```
1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}
```

In [2]:
# Initializing with SMILES
methane_smiles = 'C'
methane = rmgpy.species.Species(smiles=methane_smiles)
print(methane.to_adjacency_list())

1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}



In [3]:
methane

In [4]:
# Initializing with InChI
ethane_inchi = 'InChI=1S/C2H6/c1-2/h1-2H3'
ethane = rmgpy.species.Species(inchi=ethane_inchi)
print(ethane.to_adjacency_list())

1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 C u0 p0 c0 {1,S} {6,S} {7,S} {8,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}
6 H u0 p0 c0 {2,S}
7 H u0 p0 c0 {2,S}
8 H u0 p0 c0 {2,S}



In [5]:
ethane

In [6]:
# Initializing with adjacency list
h2o_graph = """
1 O u0 p2 c0 {2,S} {3,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
"""

h2o = rmgpy.species.Species()
h2o.from_adjacency_list(h2o_graph)
# h2o.smiles

## Species Thermo Parameters

In [7]:
print(ethane.thermo)

None


In [8]:
ethane.get_enthalpy(300)

Exception: Unable to calculate enthalpy for species '': no thermo or statmech data available.

In [ ]:
# dir(ethane)

### Thermo Database

In [9]:
# Load Thermo Database

# ref_lib = 'GRI-Mech3.0'
# ref_lib = 'JetSurF2.0'
# Load the reference thermo
library_path = "/home/moon/rmg/RMG-database/input/thermo/"
database = rmgpy.data.thermo.ThermoDatabase()
database.load(
    library_path,
    libraries = [
        'GRI-Mech3.0',
        'JetSurF2.0'
    ]
    
)

# def load(self, path, libraries=None, depository=True, surface=False):
#         """
#         Load the thermo database from the given `path` on disk, where `path`
#         points to the top-level folder of the thermo database.
#         """
#         if depository:
#             self.load_depository(os.path.join(path, 'depository'))
#         else:
#             self.depository = {}
#         self.load_libraries(os.path.join(path, 'libraries'), libraries)
#         self.load_groups(os.path.join(path, 'groups'))
#         if surface:
#             self.load_surface()

In [10]:
ethane_thermo = database.get_thermo_data(ethane)

In [11]:
print(ethane_thermo)

NASA(polynomials=[NASAPolynomial(coeffs=[4.29142,-0.00550154,5.99438e-05,-7.08466e-08,2.68686e-11,-11522.2,2.66682], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[1.07188,0.0216853,-1.00256e-05,2.21412e-09,-1.90003e-13,-11426.4,15.1156], Tmin=(1000,'K'), Tmax=(3500,'K'))], Tmin=(200,'K'), Tmax=(3500,'K'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(178.761,'J/(mol*K)'), label="""C2H6""", comment="""Thermo library: GRI-Mech3.0""")


In [22]:
spec = rmgpy.species.Species(smiles='CCCCCCCC=O')
spec_thermo = database.get_thermo_data(spec)
print(spec_thermo)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([194.109,240.217,282.84,319.787,378.36,422.07,488.325],'J/(mol*K)'), H298=(-290.712,'kJ/mol'), S298=(500.947,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(577.856,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-(Cds-O2d)CsHH) + group(Cs-CsHHH) + group(Cds-OdCsH)""")


NASA(polynomials=[NASAPolynomial(coeffs=[-2.08417,0.122535,-7.76816e-05,2.49835e-08,-3.23548e-12,-34302.2,44.226], Tmin=(298,'K'), Tmax=(1391,'K')), NASAPolynomial(coeffs=[31.9882,0.0477245,-1.62276e-05,2.50963e-09,-1.45216e-13,-46639.3,-140.504], Tmin=(1391,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(744.145,'J/(mol*K)'), label="""NC10H22""", comment="""Thermo library: JetSurF2.0""")


In [13]:
dir(database)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_adsorption_correction',
 '_add_group_thermo_data',
 '_add_poly_ring_correction_thermo_data_from_heuristic',
 '_add_polycyclic_correction_thermo_data',
 '_add_ring_correction_thermo_data_from_tree',
 '_average_children_thermo',
 '_remove_group_thermo_data',
 'binding_energies',
 'compute_group_additivity_thermo',
 'correct_binding_energy',
 'depository',
 'estimate_radical_thermo_via_hbi',
 'estimate_thermo_via_group_additivity',
 'extract_source_from_comments',
 'get_all_thermo_data',
 'get_bicyclic_correction_thermo_data_from_heuristic',
 'get_ring_groups_from_comments',
 'get_thermo_data',
 'get_t

In [ ]:
ethane_thermo.polynomials[1]
ethane.thermo = ethane_thermo

In [ ]:
ethane.get_enthalpy(300)

In [ ]:
# NASA polynomials: http://combustion.berkeley.edu/gri-mech/data/nasa_plnm.html

lib_thermo = database.get_thermo_data(ethane)
print(lib_thermo)

In [ ]:
print(lib_thermo.polynomials[0])

In [ ]:
gav_thermo = database.get_thermo_data_from_groups(ethane)

In [ ]:
all_thermo = database.get_all_thermo_data(ethane)

In [ ]:
gav_thermo

## Plot the Thermodynamic Parameters vs. Temperature

In [ ]:
def plot_thermos(thermos, labels=None):
    %matplotlib inline
    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(12, 3)
    fig.tight_layout()
    ax[0].set_xlabel('Temperature (K)')
    ax[0].set_ylabel('H (kJ / mol)')
    ax[0].set_title('Enthalpy vs. Temperature')
    ax[1].set_xlabel('Temperature (K)')
    ax[1].set_ylabel('S (kJ / mol K)')
    ax[1].set_title('Entropy vs. Temperature')
    ax[2].set_xlabel('Temperature (K)')
    ax[2].set_ylabel('Cp (kJ / mol K)')
    ax[2].set_title('Heat Capacity vs. Temperature')
    T = np.linspace(300, 3000, 1001)
    for thermo in thermos:
        H = np.zeros(len(T))
        S = np.zeros(len(T))
        Cp = np.zeros(len(T))
        for i in range(0, len(T)):
            H[i] = thermo.get_enthalpy(T[i]) / 1000.0
            S[i] = thermo.get_entropy(T[i]) / 1000.0
            Cp[i] = thermo.get_heat_capacity(T[i]) / 1000.0
        ax[0].plot(T, H)
        ax[1].plot(T, S)
        ax[2].plot(T, Cp)
    ax[0].legend(labels)
    ax[1].legend(labels)
    ax[2].legend(labels)
    plt.subplots_adjust(wspace=0.25)

In [ ]:
plot_thermos([gav_thermo, lib_thermo], labels=['GAV', 'GRI-mech3.0'])